In [1]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 59.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=7a661213f2be5214f1e094c3ac2829217f35e4bb9d98ad851db7b3d9cf3a3df1
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession
session=SparkSession.builder.appName("customer").getOrCreate()
data= session.read.csv("customer_churn.csv",header=True,inferSchema=True)
data.show(4)

+----------+------+-------------+-------+----------+------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|   MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+----------+------+-------------+-------+----------+------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+
|7590-VHVEG|Female|            0|    Yes|        No|     1|          No|No phone service|            DSL|            No|         Yes|              No|         No|    

In [4]:
data.columns

['customerID',
 'gender',
 'SeniorCitizen',
 'Partner',
 'Dependents',
 'tenure',
 'PhoneService',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod',
 'MonthlyCharges',
 'TotalCharges',
 'Churn']

In [16]:
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder
str_idx = StringIndexer(inputCols = ["gender", "Partner","MultipleLines", "PhoneService","Churn"], outputCols = ["newgender", "newPartner","newMultipleLines", "newPhoneServices","newChurn"])

In [23]:
one_hot = OneHotEncoder(inputCols = ["newgender", "newPartner","newMultipleLines", "newPhoneServices"], outputCols = ["newgender1", "newPartner1","newMultipleLines1", "newPhoneServices1"])

In [24]:
vec_ass = VectorAssembler(inputCols = ["newgender1", "newPartner1", "MonthlyCharges","newMultipleLines1", "newPhoneServices1"], outputCol = "all_features")

In [25]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(featuresCol= "all_features", labelCol = "newChurn")

In [26]:
from pyspark.ml import Pipeline
mypipeline = Pipeline(stages = [str_idx, one_hot, vec_ass, lr])

In [27]:
training, test = data.randomSplit([0.8, 0.2])

In [28]:
lr_model = mypipeline.fit(training)

In [29]:
result = lr_model.transform(test)

In [30]:
result.show(4, truncate = False)

+----------+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+-----------------------+--------------+------------+-----+---------+----------+----------------+----------------+--------+-------------+-------------+-----------------+-----------------+---------------------------+----------------------------------------+----------------------------------------+----------+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|MultipleLines   |InternetService|OnlineSecurity     |OnlineBackup       |DeviceProtection   |TechSupport        |StreamingTV        |StreamingMovies    |Contract      |PaperlessBilling|PaymentMethod          |MonthlyCharges|TotalCharges|Churn|newgender|newPartner|newMultipleLines|newPhoneServices|newChurn|newgender1   |newPartner1  |newMultipleLines1|newPhoneSe

In [31]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
eval = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol = "newChurn")

In [32]:
eval.evaluate(result)

0.685856602147736